In [1]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "./Dang_Quang_Dung_CV__Copy___Copy_.pdf"

loader = PyPDFLoader(file_path)
documents = []
for page in loader.load():
    documents.append(page)

In [2]:
documents

[Document(metadata={'source': './Dang_Quang_Dung_CV__Copy___Copy_.pdf', 'page': 0}, page_content='Dang Quang Dung\nanhdungvk01@gmail.com\n0869040236\nLinkedin\nGithub\nCau Giay - Ha Noi\nProfile\nInﬁveyears’time,IwanttobecomeanAIEngineer.MystartingpointistobeginthispositionasaSoftwareEngineer.\nEducation\n09/2019–05/2024 PostsandTelecommunicationsInstituteofTechnology .\nInformationTechnology\nGPA3.31/4.0\nResearch Interests\nNaturalLanguageProcessing(NLP),MachineLearningandDeepLearning.\nWord Experience\nMasterTraineratCentreforDevelopmentofAdvancedComputing(C-DAC),India(09/2024-\npresent)\n• ParticipateinArtiﬁcialIntelligence&DataScience:PG-DBDA-PostGraduateDiplomainBigData\nAnalytics\nAIResearcheratReseachInstituteofPostsandTelecommunications(RIPT)-PTIT(08/2024-\npresent)\n• Researchandwritingpaper\n• BuildachatbotsystemusingRAGtechniques,FunctionCallingwithLLMs\nSoftwareAIEngineeratA.I-SoftJSC(02/2023-08/2024)\n• Buildwebsitesforresearchtopics.\n• Buildachatbotsystemtosupportenroll

In [3]:
_resume_content = "\n".join([doc.page_content for doc in documents])
_resume_content

'Dang Quang Dung\nanhdungvk01@gmail.com\n0869040236\nLinkedin\nGithub\nCau Giay - Ha Noi\nProfile\nInﬁveyears’time,IwanttobecomeanAIEngineer.MystartingpointistobeginthispositionasaSoftwareEngineer.\nEducation\n09/2019–05/2024 PostsandTelecommunicationsInstituteofTechnology .\nInformationTechnology\nGPA3.31/4.0\nResearch Interests\nNaturalLanguageProcessing(NLP),MachineLearningandDeepLearning.\nWord Experience\nMasterTraineratCentreforDevelopmentofAdvancedComputing(C-DAC),India(09/2024-\npresent)\n• ParticipateinArtiﬁcialIntelligence&DataScience:PG-DBDA-PostGraduateDiplomainBigData\nAnalytics\nAIResearcheratReseachInstituteofPostsandTelecommunications(RIPT)-PTIT(08/2024-\npresent)\n• Researchandwritingpaper\n• BuildachatbotsystemusingRAGtechniques,FunctionCallingwithLLMs\nSoftwareAIEngineeratA.I-SoftJSC(02/2023-08/2024)\n• Buildwebsitesforresearchtopics.\n• Buildachatbotsystemtosupportenrollment(End-to-End)usingRasaandBotpressFramework.\n• Buildacrawldatamodule,dataanalysisusingBERTopic

In [4]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2")

# llm.invoke("The first man on the moon was ...")

# LLM

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [7]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import pydantic
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="llama3.2")


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

# chain.invoke({"query": joke_query})

In [8]:
from typing import List, Optional
from pydantic import BaseModel, EmailStr, Field, root_validator, field_validator

In [9]:
# Pydantic model for extracting education
class Education(BaseModel):
    institution: Optional[str] = Field(
        None, description="The name of the educational institution"
    )
    degree: Optional[str] = Field(
        None, description="The degree or qualification earned"
    )
    graduation_date: Optional[str] = Field(
        None, description="The graduation date (e.g., 'YYYY-MM')"
    )
    details: Optional[List[str]] = Field(
        None,
        description="Additional details about the education (e.g., coursework, achievements)",
    )

    @field_validator("details", mode="before")
    def validate_details(cls, v):
        if isinstance(v, str) and v.lower() == "n/a":
            return []
        elif not isinstance(v, list):
            return []
        return v


# Pydantic model for extracting experience
class Experience(BaseModel):
    company: Optional[str] = Field(
        None, description="The name of the company or organization"
    )
    location: Optional[str] = Field(
        None, description="The location of the company or organization"
    )
    role: Optional[str] = Field(
        None, description="The role or job title held by the candidate"
    )
    start_date: Optional[str] = Field(
        None, description="The start date of the job (e.g., 'YYYY-MM')"
    )
    end_date: Optional[str] = Field(
        None,
        description="The end date of the job or 'Present' if ongoing (e.g., 'MM-YYYY')",
    )
    responsibilities: Optional[List[str]] = Field(
        None, description="A list of responsibilities and tasks handled during the job"
    )

    @field_validator("responsibilities", mode="before")
    def validate_responsibilities(cls, v):
        if isinstance(v, str) and v.lower() == "n/a":
            return []
        elif not isinstance(v, list):
            return []
        return v


# Main Pydantic class ensapsulating education and epxerience classes with other information
class Candidate(BaseModel):
    name: Optional[str] = Field(None, description="The full name of the candidate")
    email: Optional[EmailStr] = Field(None, description="The email of the candidate")
    age: Optional[int] = Field(None, description="The age of the candidate.")
    skills: Optional[List[str]] = Field(
        None, description="A list of high-level skills possessed by the candidate."
    )
    experience: Optional[List[Experience]] = Field(
        None,
        description="A list of experiences detailing previous jobs, roles, and responsibilities",
    )
    education: Optional[List[Education]] = Field(
        None,
        description="A list of educational qualifications of the candidate including degrees, institutions studied in, and dates of start and end.",
    )

class CandidateExperience(BaseModel):
    experience: Optional[List[Experience]] = Field(
        None,
        description="A list of experiences detailing previous jobs, roles, and responsibilities",
    )
    

class CandidateEducation(BaseModel):
    education: Optional[List[Education]] = Field(
        None,
        description="A list of educational qualifications of the candidate including degrees, institutions studied in, and dates of start and end.",
    )


    @root_validator(pre=True)
    def handle_invalid_values(cls, values):
        for key, value in values.items():
            if isinstance(value, str) and value.lower() in {"n/a", "none", ""}:
                values[key] = None
        return values

C:\Users\Admin\AppData\Local\Temp\ipykernel_26588\3241683779.py:88: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @root_validator(pre=True)


In [10]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."
resume = "Dang Quang Dung\nanhdungvk01@gmail.com\n0869040236\nLinkedin\nGithub\nCau Giay - Ha Noi\nProfile\nInﬁveyears’time,IwanttobecomeanAIEngineer.MystartingpointistobeginthispositionasaSoftwareEngineer.\nEducation\n09/2019–05/2024 PostsandTelecommunicationsInstituteofTechnology .\nInformationTechnology\nGPA3.31/4.0\nResearch Interests\nNaturalLanguageProcessing(NLP),MachineLearningandDeepLearning.\nWord Experience\nMasterTraineratCentreforDevelopmentofAdvancedComputing(C-DAC),India(09/2024-\npresent)\n•ParticipateinArtiﬁcialIntelligence&DataScience:PG-DBDA-PostGraduateDiplomainBigData\nAnalytics\nAIResearcheratReseachInstituteofPostsandTelecommunications(RIPT)-PTIT(08/2024-\npresent)\n•Researchandwritingpaper\n•BuildachatbotsystemusingRAGtechniques,FunctionCallingwithLLMs\nSoftwareAIEngineeratA.I-SoftJSC(02/2023-08/2024)\n•Buildwebsitesforresearchtopics.\n•Buildachatbotsystemtosupportenrollment(End-to-End)usingRasaandBotpressFramework.\n•Buildacrawldatamodule,dataanalysisusingBERTopicforsocialnetworklisteningsystem.\n•ResearchpaperaboutAIandNaturalLanguageProcessingandapplytousedsoftwareproducts.\n•Buildachatbotsystemtosupportenrollment(End-to-End)withLLMsusingRAG\n(Retrieval-AugmentedGeneration)techniqueswithLangChainandLlamaIndexFramework.\nSkills\nLanguages English,Vietnamese(nativespeaker)\nCoding Python,Linux,SQL,FastAPI,Docker,...\nDatabases My sql,MongoDB,FAISS,Chroma...\nWebDev Html, css ,JavaScript,Reactjs,Antdesign,MUI...\nSkills (continued)\nOpen-sourceNLPTools Transformers(HuggingFace),spaCy,scikit-learn,Gensim,NLTK,BERTopic,\nRasa,Botpress,Langchain,LlamaIndex...\nDeepLearningFrameworks TensorFlow,Keras,PyTorch\nMisc. Research Skills, Problem Solving, Report Writing, Teamwork and Indepen-\ndently,DataStructuresandAlgorithms,DataVisualization(Tableau,Excel)\nScientific research achievements\nPublished scientific articles\n2024 An intent-ﬁltered retrieval-augmented generation chatbot for university admissions in\nVietnam .InternationalConferenceOnAdvancedTechnologiesForCommunications.\nMiscellaneous Experience\nCertification\n11/2021 APPLICATIONSOFALGORITHM .byPostsandTelecommunicationsInstituteofTechnology\nandSamsungElectronicsVietnamCompanyLimited.\n09/2022 Toeic605 .byIIGVietNam."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Candidate)

prompt = PromptTemplate(
    template=""" You are an expert in extract and analyzing resumes. Use formart of the JSON Candidate schema below to extract relevant information json.
    \n{format_instructions}\n{resume}\n
    NOTE: Skills must be is list string or None\n
    Answer the user json resume\n""",
    input_variables=["resume"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

# res = chain.invoke({"resume": resume})

In [106]:
res

{'$defs': {'Education': {'properties': {'institution': {'anyOf': [{'type': 'string'},
      {'type': 'null'}],
     'default': None},
    'degree': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None},
    'graduation_date': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None},
    'details': {'anyOf': [{'items': {'type': 'string'}, 'type': 'array'},
      {'type': 'null'}],
     'default': None}},
   'title': 'Education',
   'type': 'object'},
  'Experience': {'properties': {'company': {'anyOf': [{'type': 'string'},
      {'type': 'null'}],
     'default': None},
    'location': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None},
    'role': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None},
    'start_date': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None},
    'end_date': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None},
    'responsibilities': {'anyOf': [{'items

In [98]:
# Set up a parser + inject instructions into the prompt template.
parser_experience = JsonOutputParser(pydantic_object=CandidateExperience)

prompt_experience = PromptTemplate(
    template=""" You are an expert in extract and analyzing resumes. Use formart of the JSON CandidateExperience schema below to extract relevant information json.
    \n{format_instructions}\n{resume}\n
    Answer the user json resume\n""",
    input_variables=["resume"],
    partial_variables={"format_instructions": parser_experience.get_format_instructions()},
)

chain = prompt_experience | model | parser_experience

res_experience = chain.invoke({"resume": resume})

In [99]:
res_experience

{'$defs': {'Experience': {'properties': {'company': {'anyOf': [{'type': 'string'},
      {'type': 'null'}],
     'default': None,
     'description': 'The name of the company or organization',
     'title': 'Company'},
    'location': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'description': 'The location of the company or organization',
     'title': 'Location'},
    'role': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'description': 'The role or job title held by the candidate',
     'title': 'Role'},
    'start_date': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'description': "The start date of the job (e.g., 'YYYY-MM')",
     'title': 'Start Date'},
    'end_date': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'description': "The end date of the job or 'Present' if ongoing (e.g., 'MM-YYYY')",
     'title': 'End Date'},
    'responsibilities': {'anyOf': [

In [16]:
response = {'name': 'Dang Quang Dung',
 'email': 'anhdungvk01@gmail.com',
 'age': 29,
 'skills': {'Open-source NLP Tools': ['Transformers(HuggingFace)',
   'spaCy',
   'scikit-learn',
   'Gensim',
   'NLTK',
   'BERTopic'],
  'Deep Learning Frameworks': ['TensorFlow', 'Keras', 'PyTorch']},
 'experience': [{'company': 'CentreforDevelopmentofAdvancedComputing (C-DAC), India',
   'location': 'Cau Giay - Ha Noi',
   'role': 'Master Trainer',
   'start_date': None,
   'end_date': None,
   'responsibilities': ['Participate in Artificial Intelligence & Data Science: PG-DBDA-PostGraduateDiplomainBigData Analytics']},
  {'company': 'ResearchInstituteofPostsandTelecommunications (RIPT)-PTIT',
   'location': None,
   'role': 'AI Researcher',
   'start_date': None,
   'end_date': None,
   'responsibilities': ['Research and writing paper',
    'Build a chatbot system using RAG techniques, Function Calling with LLMs']},
  {'company': 'A.I-SoftJSC',
   'location': None,
   'role': 'Software AI Engineer',
   'start_date': '02/2023',
   'end_date': '08/2024',
   'responsibilities': ['Build websites for research topics.',
    'Build a chatbot system to support enrollment (End-to-End) using Rasa and Botpress Framework.']}],
 'education': [{'institution': 'PostsandTelecommunicationsInstituteofTechnology',
   'degree': 'Information Technology',
   'graduation_date': '09/2019–05/2024',
   'details': ['GPA3.31/4.0',
    'Research Interests: NaturalLanguageProcessing(NLP),MachineLearningandDeepLearning.']}],
 'scientific research achievements': {'Published scientific articles': ['2024 An intent-ﬁltered retrieval-augmented generation chatbot for university admissions in Vietnam. InternationalConferenceOnAdvancedTechnologiesForCommunications.']}}

In [53]:
# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Candidate)

prompt = PromptTemplate(
    template=""" You are an expert in repair invalid JSON by errors. Use the JSON formart correct to repair invalid json.
    \n{format_instructions}\n, Error: {error} and json invalid: {json_invalid}\n
    Answer the JSON format correct\n""",
    input_variables=["json_invalid", "error"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

In [74]:
import json
# parsed_data = json.loads(response)
try:
    # res = chain.invoke({"json_invalid": response})
    parsed_data = res
    response = Candidate.model_validate(parsed_data)
except Exception as e:
    error = e
    # res_repair = chain.invoke({"json_invalid": response, "error": error})
    print(f"Error parsing response: {str(e)}")
finally:
    response = Candidate.model_validate(parsed_data)

In [75]:
response

Candidate(name=None, email=None, age=None, skills=None, experience=None, education=None)

In [54]:
res_repair = chain.invoke({"json_invalid": response, "error": str(error)})

In [55]:
res_repair

{'name': 'Dang Quang Dung',
 'email': 'anhdungvk01@gmail.com',
 'age': 29,
 'skills': ['Open-source NLP Tools: Transformers(HuggingFace), spaCy, scikit-learn, Gensim, NLTK, BERTopic',
  'Deep Learning Frameworks: TensorFlow, Keras, PyTorch'],
 'experience': [{'company': 'CentreforDevelopmentofAdvancedComputing (C-DAC), India',
   'location': 'Cau Giay - Ha Noi',
   'role': 'Master Trainer',
   'start_date': None,
   'end_date': None,
   'responsibilities': ['Participate in Artificial Intelligence & Data Science: PG-DBDA-PostGraduateDiplomainBigData Analytics']},
  {'company': 'ResearchInstituteofPostsandTelecommunications (RIPT)-PTIT',
   'location': None,
   'role': 'AI Researcher',
   'start_date': None,
   'end_date': None,
   'responsibilities': ['Research and writing paper',
    'Build a chatbot system using RAG techniques, Function Calling with LLMs']},
  {'company': 'A.I-SoftJSC',
   'location': None,
   'role': 'Software AI Engineer',
   'start_date': '02/2023',
   'end_date': 

Candidate(name='Dang Quang Dung', email='anhdungvk01@gmail.com', age=29, skills=['Open-source NLP Tools: Transformers(HuggingFace), spaCy, scikit-learn, Gensim, NLTK, BERTopic', 'Deep Learning Frameworks: TensorFlow, Keras, PyTorch'], experience=[Experience(company='CentreforDevelopmentofAdvancedComputing (C-DAC), India', location='Cau Giay - Ha Noi', role='Master Trainer', start_date=None, end_date=None, responsibilities=['Participate in Artificial Intelligence & Data Science: PG-DBDA-PostGraduateDiplomainBigData Analytics']), Experience(company='ResearchInstituteofPostsandTelecommunications (RIPT)-PTIT', location=None, role='AI Researcher', start_date=None, end_date=None, responsibilities=['Research and writing paper', 'Build a chatbot system using RAG techniques, Function Calling with LLMs']), Experience(company='A.I-SoftJSC', location=None, role='Software AI Engineer', start_date='02/2023', end_date='08/2024', responsibilities=['Build websites for research topics.', 'Build a chatbo

In [ ]:
chain.invoke({"query": joke_query})

In [11]:
parser = JsonOutputParser(pydantic_object=Candidate)

prompt = PromptTemplate(
    template=""" You are an expert in analyzing resumes. Use the following JSON schema to extract relevant information.
    Answer the user resume.\n{format_instructions}\n{resume}\n""",
    input_variables=["resume"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
response = None
is_error = False
errors = None
chain = prompt | llm | parser
try:
    response = chain.invoke({"resume": resume})
    if not response or not response:
        raise ValueError("Failed to get a response from LLM.")

    parsed_data = response
    response = Candidate.model_validate(parsed_data)
except Exception as e:
    print(f"Error parsing response: {str(e)}")
    errors = e

    raise ValueError(
        "Failed to extract insights. Please ensure the resume and query engine are properly configured."
    )


In [12]:
parsed_data

{'name': 'Dang Quang Dung',
 'email': 'anhdungvk01@gmail.com',
 'age': None,
 'skills': ['English',
  'Vietnamese (native speaker)',
  'Python',
  'Linux',
  'SQL',
  'FastAPI',
  'Docker',
  'MySQL',
  'MongoDB',
  'FAISS',
  'Chroma',
  'HTML',
  'CSS',
  'JavaScript',
  'ReactJS',
  'Ant Design',
  'MUI',
  'Transformers (Hugging Face)',
  'spaCy',
  'scikit-learn',
  'Gensim',
  'NLTK',
  'BERTopic',
  'Rasa',
  'Botpress',
  'Langchain',
  'LlamaIndex',
  'TensorFlow',
  'Keras',
  'PyTorch',
  'Research Skills',
  'Problem Solving',
  'Report Writing',
  'Teamwork',
  'Data Structures and Algorithms',
  'Data Visualization (Tableau, Excel)'],
 'experience': [{'company': 'Centre for Development of Advanced Computing (C-DAC)',
   'location': 'India',
   'role': 'Master Trainer',
   'start_date': '2024-09',
   'end_date': 'Present',
   'responsibilities': ['Participate in Artificial Intelligence & Data Science: PG-DBDA - Post Graduate Diploma in Big Data Analytics']},
  {'company': 

In [13]:
response

Candidate(name='Dang Quang Dung', email='anhdungvk01@gmail.com', age=None, skills=['English', 'Vietnamese (native speaker)', 'Python', 'Linux', 'SQL', 'FastAPI', 'Docker', 'MySQL', 'MongoDB', 'FAISS', 'Chroma', 'HTML', 'CSS', 'JavaScript', 'ReactJS', 'Ant Design', 'MUI', 'Transformers (Hugging Face)', 'spaCy', 'scikit-learn', 'Gensim', 'NLTK', 'BERTopic', 'Rasa', 'Botpress', 'Langchain', 'LlamaIndex', 'TensorFlow', 'Keras', 'PyTorch', 'Research Skills', 'Problem Solving', 'Report Writing', 'Teamwork', 'Data Structures and Algorithms', 'Data Visualization (Tableau, Excel)'], experience=[Experience(company='Centre for Development of Advanced Computing (C-DAC)', location='India', role='Master Trainer', start_date='2024-09', end_date='Present', responsibilities=['Participate in Artificial Intelligence & Data Science: PG-DBDA - Post Graduate Diploma in Big Data Analytics']), Experience(company='Research Institute of Posts and Telecommunications (RIPT)-PTIT', location=None, role='AI Research

In [18]:
insights = response

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

In [17]:
from langchain_community.vectorstores import FAISS
job_index = FAISS.load_local(
    folder_path='../Multiple_models/job_index_3k',
    embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5"),
    allow_dangerous_deserialization=True,
)

c:\Users\Admin\anaconda3\envs\langchain-chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
education = (
    [edu.degree for edu in insights.education]
    if insights.education
    else []
)
skills = insights.skills or []
experience = (
    [exp.role for exp in insights.experience]
    if insights.experience
    else []
)
print(education)
print("=========")
print(skills)
print("========")
print(experience)

['Information Technology']
['English', 'Vietnamese (native speaker)', 'Python', 'Linux', 'SQL', 'FastAPI', 'Docker', 'MySQL', 'MongoDB', 'FAISS', 'Chroma', 'HTML', 'CSS', 'JavaScript', 'ReactJS', 'Ant Design', 'MUI', 'Transformers (Hugging Face)', 'spaCy', 'scikit-learn', 'Gensim', 'NLTK', 'BERTopic', 'Rasa', 'Botpress', 'Langchain', 'LlamaIndex', 'TensorFlow', 'Keras', 'PyTorch', 'Research Skills', 'Problem Solving', 'Report Writing', 'Teamwork', 'Data Structures and Algorithms', 'Data Visualization (Tableau, Excel)']
['Master Trainer', 'AI Researcher', 'Software AI Engineer']


In [22]:
query = f"Education: {', '.join(education)}; Skills: {', '.join(skills)}; Experience: {', '.join(experience)}"
retriever = job_index.as_retriever(similarity_top_k=3)
matching_jobs = retriever.invoke(query)
matching_jobs

[Document(metadata={'source': None, 'seq_num': 67, 'id': 136592, 'title': 'Receptionist', 'branch': None, 'company': 'OfficeTeam', 'location': 'San Bruno, CA', 'salaryRange': None, 'employmentType': 'Seasonal/Temp', 'datePosted': '2014-12-23 13:18:35 UTC'}, page_content="Receptionist, Seasonal/Temp, Salary ($ per hour/ day) nan, in San Bruno, CA, education requiredNot Specified, Company OfficeTeam\n\n Ref ID: 00430-135580Classification: Receptionist/SwitchboardCompensation: $14.00 to $15.00 per hourOfficeTeam is the world's leading staffing service specializing in the placement of highly skilled office and administrative support professionals on a temporary and temporary to full-time basis. OfficeTeam is recruiting for Super Star Front Desk Coordinator that has an interest in Education for a private school! Our client is looking for a Front Desk Coordinator to be the first point of contact for their school. You will be helping with greeting parents, taking care of enrollment papers, ca

In [26]:
retrieved_context = "\n\n".join(
    [match.page_content for match in matching_jobs]
)
candidate_details = f"Education: {', '.join(education)}; Skills: {', '.join(skills)}; Experience: {', '.join(experience)}"

In [ ]:
if is_error:
    prompt = PromptTemplate(
        template=""" You are an expert in repair invalid JSON by errors. Use the JSON formart correct to repair invalid json.
        \n{format_instructions}\n, Error: {error} and json invalid: {json_invalid}\n
        Answer the JSON format correct\n""",
        input_variables=["json_invalid", "error"],
        partial_variables={
            "format_instructions": parser.get_format_instructions()
        },
    )

    chain = prompt | llm | parser
    res_repair = chain.invoke({"json_invalid": response, "error": str(errors)})
    # response = Candidate.model_validate(res_repair)
    response = res_repair
response

In [23]:
from typing import Union
from langchain_core.retrievers import BaseRetriever
from pydantic import BaseModel, Field, ConfigDict
from langchain_core.output_parsers import StrOutputParser

class RAGStringQueryEngine(BaseModel):
    """
    Custom Query Engine for Retrieval-Augmented Generation (fetching matching job recommendations).
    """

    retriever: BaseRetriever
    llm: Union[ChatOpenAI, OllamaLLM]
    qa_prompt: PromptTemplate

    # Allow arbitrary types
    model_config = ConfigDict(arbitrary_types_allowed=True)

    def custom_query(self, candidate_details: str, retrieved_jobs: str):
        query_str = self.qa_prompt.format(
            query_str=candidate_details, context_str=retrieved_jobs
        )
        output_parser = StrOutputParser()
        llm_chain = self.qa_prompt | self.llm | output_parser
        if isinstance(self.llm, ChatOpenAI):
            # OpenAI-specific query handling
            response = llm_chain.invoke(
                {"query_str": candidate_details, "context_str": retrieved_jobs}
            )
        elif isinstance(self.llm, OllamaLLM):
            # Ollama-specific query handling
            response = llm_chain.invoke(
                {"query_str": candidate_details, "context_str": retrieved_jobs}
            )
        else:
            raise ValueError("Unsupported LLM type. Please use OpenAI or Ollama.")

        return str(response)

In [27]:
rag_engine = RAGStringQueryEngine(
    retriever=job_index.as_retriever(),
    # llm=analyzer.llm,  # This can be OpenAI or Ollama
    llm=llm,  # This can be OpenAI or Ollama
    qa_prompt=PromptTemplate(
        input_variables=["query_str", "context_str"],
        template="""\
        You are expert in analyzing resumes, based on the following candidate details and job descriptions:
        Candidate Details:
        ---------------------
        {query_str}
        ---------------------
        Job Descriptions:
        ---------------------
        {context_str}
        ---------------------
        Provide a concise list of the matching jobs. For each matching job, mention job-related details such as 
        company, brief job description, location, employment type, salary range, URL for each suggestion, and a brief explanation of why the job matches the candidate's profile.
        Be critical in matching profile with the jobs. Thoroughly analyze education, skills, and experience to match jobs.  
        Do not explain why the candidate's profile does not match with the other jobs. Do not include any summary. Order the jobs based on their relevance. 
        Answer: 
        """,
    ),
)

llm_response = rag_engine.custom_query(
    candidate_details=candidate_details,
    retrieved_jobs=retrieved_context,
)

In [28]:
llm_response

"1. **Job Title:** Human Resources Recruiter  \n   **Company:** OfficeTeam  \n   **Job Description:** Responsible for gathering job description details, posting jobs, screening resumes, actively recruiting, phone screening candidates, scheduling interviews, generating offer letters, and assisting with onboarding.  \n   **Location:** Milwaukee, WI  \n   **Employment Type:** Seasonal/Temp  \n   **Salary Range:** $16.15 to $18.70 per hour  \n   **URL:** [Apply Here](mailto:OfficeTeam@414-271-4003)  \n   **Explanation:** The candidate has strong communication skills and experience in teamwork and problem-solving, which are essential for recruiting. Their research skills and ability to work with various technologies could aid in sourcing candidates effectively.\n\n2. **Job Title:** Receptionist  \n   **Company:** OfficeTeam  \n   **Job Description:** First point of contact for a private school, responsible for greeting parents, managing enrollment papers, calendar management, and answering 

In [33]:
[print(i) for i in llm_response.split('\n')]


1. **Job Title:** Human Resources Recruiter  
   **Company:** OfficeTeam  
   **Job Description:** Responsible for gathering job description details, posting jobs, screening resumes, actively recruiting, phone screening candidates, scheduling interviews, generating offer letters, and assisting with onboarding.  
   **Location:** Milwaukee, WI  
   **Employment Type:** Seasonal/Temp  
   **Salary Range:** $16.15 to $18.70 per hour  
   **URL:** [Apply Here](mailto:OfficeTeam@414-271-4003)  
   **Explanation:** The candidate has strong communication skills and experience in teamwork and problem-solving, which are essential for recruiting. Their research skills and ability to work with various technologies could aid in sourcing candidates effectively.

2. **Job Title:** Receptionist  
   **Company:** OfficeTeam  
   **Job Description:** First point of contact for a private school, responsible for greeting parents, managing enrollment papers, calendar management, and answering phones.  
  

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

1. **Job Title:** Human Resources Recruiter  
   **Company:** OfficeTeam  
   **Job Description:** Responsible for gathering job description details, posting jobs, screening resumes, actively recruiting, phone screening candidates, scheduling interviews, generating offer letters, and assisting with onboarding.  
   **Location:** Milwaukee, WI  
   **Employment Type:** Seasonal/Temp  
   **Salary Range:** $16.15 to $18.70 per hour  
   **URL:** [Apply Here](mailto:OfficeTeam@414-271-4003)  
   **Explanation:** The candidate has strong communication skills and experience in teamwork and problem-solving, which are essential for recruiting. Their research skills and ability to work with various technologies could aid in sourcing candidates effectively.

2. **Job Title:** Receptionist  
   **Company:** OfficeTeam  
   **Job Description:** First point of contact for a private school, responsible for greeting parents, managing enrollment papers, calendar management, and answering phones.  
   **Location:** San Bruno, CA  
   **Employment Type:** Seasonal/Temp  
   **Salary Range:** $14.00 to $15.00 per hour  
   **URL:** [Apply Here](mailto:OfficeTeam@ClickHereToEmailYourResumé)  
   **Explanation:** The candidate's strong verbal and written communication skills, along with their ability to multitask, align well with the requirements of a receptionist role. Their experience in customer service and teamwork would be beneficial in this position.

3. **Job Title:** Receptionist  
   **Company:** Mind of Beauty Day Spa  
   **Job Description:** Act as a receptionist and support staff as needed.  
   **Location:** Los Altos, CA  
   **Employment Type:** Part-Time  
   **Salary Range:** Not specified  
   **URL:** [Apply Here](mailto:MindOfBeautyDaySpa@Apply)  
   **Explanation:** The candidate's communication skills and ability to work in a team environment make them suitable for a receptionist role. Although the job requires fluency in Mandarin, the candidate's bilingual skills in Vietnamese and English may still be advantageous in a diverse workplace.

In [ ]:
llm_response = "1. **Job Title:** Human Resources Recruiter  \n   **Company:** OfficeTeam  \n   **Job Description:** Responsible for gathering job description details, posting jobs, screening resumes, actively recruiting, phone screening candidates, scheduling interviews, generating offer letters, and assisting with onboarding.  \n   **Location:** Milwaukee, WI  \n   **Employment Type:** Seasonal/Temp  \n   **Salary Range:** $16.15 to $18.70 per hour  \n   **URL:** [Apply Here](mailto:OfficeTeam@414-271-4003)  \n   **Explanation:** The candidate has strong communication skills and experience in teamwork and problem-solving, which are essential for recruiting. Their research skills and ability to work with various technologies could aid in sourcing candidates effectively.\n\n2. **Job Title:** Receptionist  \n   **Company:** OfficeTeam  \n   **Job Description:** First point of contact for a private school, responsible for greeting parents, managing enrollment papers, calendar management, and answering phones.  \n   **Location:** San Bruno, CA  \n   **Employment Type:** Seasonal/Temp  \n   **Salary Range:** $14.00 to $15.00 per hour  \n   **URL:** [Apply Here](mailto:OfficeTeam@ClickHereToEmailYourResumé)  \n   **Explanation:** The candidate's strong verbal and written communication skills, along with their ability to multitask, align well with the requirements of a receptionist role. Their experience in customer service and teamwork would be beneficial in this position.\n\n3. **Job Title:** Receptionist  \n   **Company:** Mind of Beauty Day Spa  \n   **Job Description:** Act as a receptionist and support staff as needed.  \n   **Location:** Los Altos, CA  \n   **Employment Type:** Part-Time  \n   **Salary Range:** Not specified  \n   **URL:** [Apply Here](mailto:MindOfBeautyDaySpa@Apply)  \n   **Explanation:** The candidate's communication skills and ability to work in a team environment make them suitable for a receptionist role. Although the job requires fluency in Mandarin, the candidate's bilingual skills in Vietnamese and English may still be advantageous in a diverse workplace."

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

In [38]:
import sys
sys.path.insert(0, '/Job-recommend/CV-Analyzer-Job-Recommender/Multiple models/')

In [1]:
from utils import get_embedding

c:\Users\Admin\anaconda3\envs\langchain-chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from typing import *
import openai
import os
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from dotenv import load_dotenv

load_dotenv()

client = openai.OpenAI(api_key = os.getenv("OPENAI_API_KEY"))
# openai.api_key = os.environ.get("OPENAI_API_KEY")

class Structure(BaseModel):
    intro: Optional[str] = Field(..., description="Introduction section of the answer")
    personal: Optional[str] = Field(
        ..., description="Personal opinion section of the answer"
    )
    conclusion: str = Field(..., description="Short conclusion section of the answer")

    def structure_query(self):
        query = ""
        if self.intro is not None:
            query += f"Introduction: {self.intro}\n"
        if self.personal is not None:
            query += f"Personal opinion: {self.personal}\n"
        if self.conclusion is not None:
            query += f"Short conclusion: {self.conclusion}\n"
        return query


def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model="gpt-4o-mini", messages=messages, temperature=0
    )
    return response.choices[0].message.content


def stup_format_instructions():
    response_schemas = [
        ResponseSchema(
            name="structure",
            description="""Structure validation results:
            - "intro_match": Whether the introduction matches the structure (boolean).
            - "personal_match": Whether the personal opinion matches (boolean).
            - "conclusion_match": Whether the short conclusion matches (boolean).
            - "missing_phrases": List of required phrases that are missing (list).""",
        ),
        ResponseSchema(
            name="content",
            description="""Content validation results:
            - "relevance": Relevance score (float between 0 and 1).
            - "logic": Logic score (float between 0 and 1).""",
        ),
        ResponseSchema(
            name="feedback",
            description="""Feedback details:
            - "intro_feedback": Feedback for the introduction (string).
            - "personal_feedback": Feedback for the personal opinion section (string).
            - "conclusion_feedback": Feedback for the conclusion (string).
            - "suggestions": List of actionable suggestions for improvement (list).""",
        ),
    ]

    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()

    return output_parser, format_instructions


def stup_prompt():
    template = """
    You are an english tutor. With a question {question_query}, you ask student to write a paragraph including 3 sections (Intro, Personal Opinion, Conclusion) following a strictly structure:
    "Intro: Oh yes, definitely! There's no denying that  + (positive aspect of something) + People enjoy it because' + (explanations),
    Personal Opinion: However, for me personally, it depends on +  .... + For example, + (specific example(s) and explanations),
    Conclusion: Like' + ...",
    With a student's answer {student_answer}, you analyze it based on the following criteria:
        - Structure requirements:
            + Each part must follow the specified structure and include required words.
            + Provide detailed feedback for missing part:
                - Intro: "You missed the introduction. Start with something like..."
                - Opinion: "You forgot to include a specific example. Try again with..."
                - Conclustion: "Don't forget to finish with a short conclusion, like... to wrap up your answer."
        - Content relevance:
                + Evaluate how well the answer addresses the question: {question_query}."
    Return the analysis result strictly in the following plaintext format."""

    prompt = PromptTemplate(
        template=template,
        input_variables=[
            "question_query",
            "structure_query",
            "student_answer",
            "format_instructions",
        ],
    )
    return prompt


output_parser, format_instructions = stup_format_instructions()
prompt = stup_prompt()


def response(question: AnyStr, answer: AnyStr, structure: Structure):
    query = prompt.format(
        question_query=question,
        structure_query=structure.structure_query(),
        student_answer=answer,
        format_instructions=format_instructions,
    )
    return get_completion(query)


def response_v2(
    question: AnyStr,
    answer: AnyStr,
    structure_intro: AnyStr,
    structure_personal: AnyStr,
    structure_conclusion: AnyStr,
):
    structure = Structure(
        intro=structure_intro,
        personal=structure_personal,
        conclusion=structure_conclusion,
    )
    query = prompt.format(
        question_query=question,
        structure_query=structure.structure_query(),
        student_answer=answer,
        format_instructions=format_instructions,
    )
    return get_completion(query)  

In [2]:
original_text = """
    Oh yes, definitely! There's no denying that living in a noisy place can be vibrant and full of energy. People enjoy it because it keeps them active and engaged.
    
    However, for me personally, it depends on the situation. For example, if I am studying, I prefer a quiet place to concentrate. However, during a festival, I enjoy staying in a lively and noisy environment to feel the energy and excitement.
   
    Like the buzz in a busy café that merges seamlessly into the surroundings, becoming a soundtrack to efficiency and communal activity.
    """
structure = Structure(
    intro="'Oh yes, definitely! There's no denying that'  + (positive aspect of something) + 'People enjoy it because' + (explanations)",
    personal="'However, for me personally, it depends on' +  .... + 'For example,' + (specific example(s))",
    conclusion="'Like' + ...",
)
questions = "Do you like to stay in a place with a lot of noise?"

answer = """
Oh yes, definitely! There's no denying that living in a place with a lot of noise can make people feel energetic and lively. People enjoy it because it creates a sense of vibrancy and excitement.

However, for me personally, it depends on the situation. For example, I enjoy the buzz of a noisy café when catching up with friends, as it adds to the fun. But when it comes to studying or working, I prefer a quieter environment to help me concentrate better.

Like I said, it really depends on the context and my mood at the time."""

answer2 = """I prefer a quiet place. Noise can be very annoying and I don't like being in loud environments. Sometimes, I go to places with more people, but usually, I avoid it."""
print(response(question=questions, answer=answer, structure=structure))
print("================================================================")
print(response(question=questions, answer=answer2, structure=structure))
print("================================================================")
print(response(question=questions, answer=original_text, structure=structure))

Structure requirements:
- Intro: "You followed the introduction structure well, starting with 'Oh yes, definitely!' and providing a positive aspect of living in a noisy place. However, you could enhance it by adding a more specific explanation of why people enjoy the vibrancy and excitement."
- Opinion: "You included a personal opinion, but you could strengthen it by providing a more detailed specific example. For instance, you could elaborate on how the atmosphere of a noisy café enhances your experience with friends."
- Conclusion: "You provided a conclusion, but it could be more definitive. Try finishing with a statement that summarizes your overall feelings about noise in different contexts, like 'In conclusion, while I appreciate the energy of noise in social settings, I also value quietness for focus.'"

Content relevance:
- Your answer addresses the question well, as you express both a general appreciation for noise and your personal preferences. You effectively convey that your

In [1]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List, Optional
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(temperature=0)

class ContentScore(BaseModel):
    relevance: Optional[float] = Field(
        None, description="Relevance score (float between 0 and 1)."
    )
    logic: Optional[float] = Field(
        None, description="Logic score (float between 0 and 1)."
    )

# Define your desired data structure.
class AIAnswer(BaseModel):
    content_score: Optional[ContentScore] = Field(
        None, description="Content validation results. If the answer is relevant and logical, provides a relevance score and a logic score. For example: 'Relevance: 0.8, Logic: 0.9.")
    ai_feedback: Optional[str] = Field(
        None, description="If the student answers with the correct structure, just shows positive feedback such as: 'Great job! You followed the structure perfectly!' or 'Good effort! Your answer follows the structure well, but pay attention to small grammatical mistakes and word choice. Try using more precise vocabulary and check for subject-verb agreement.' or 'Your response is clear and understandable. However, avoid minor errors in grammar, like 'make' instead of 'makes.' Also, add more specific examples for a stronger answer.' or 'Great job with the structure! To improve, focus on expanding your vocabulary and ensuring your grammar is accurate.'")
    missing_example: Optional[str] = Field(
        None, description="If any part of the structure is missing, displays detailed suggestions to Missing example with format like: 'You forgot to include a specific example. Try again with 'For example, + explanation'")
    missing_intro: Optional[str] = Field(
        None, description="If any part of the structure is missing, displays detailed suggestions to Missing opening sentence with format like: 'You missed the introduction. Start with something like 'Oh yes, definitely! There's no denying that….'")
    missing_a_short_concluding_sentence: Optional[str] = Field(
        None, description="If any part of the structure is missing, displays detailed suggestions to Missing a short concluding sentence with format like: 'Don't forget to finish with a short conclusion, like 'Like…' to wrap up your answer.'")
    grammar_errors: Optional[List[str]] = Field(
        None, description="If the verb tense is grammatically incorrect, provides feedback on grammar errors. For instance: 'Your grammar is mostly correct, but you made a small error in verb tense. For example, 'It depend' should be 'It depends.'")
    agreement_between_subject_and_verb_errors: Optional[List[str]] = Field(
        None, description="If there is an error in the agreement between subject and verb, provides feedback on agreement between subject and verb. For instance: 'Be careful with subject-verb agreement. For example, 'People enjoys' should be 'People enjoy.'")
    sentence_structure_diversity_errors: Optional[List[str]] = Field(
        None, description="If the sentence structure is not varied, provides feedback on sentence structure diversity. For instance: 'You could improve sentence variety by using different structures. Instead of 'People enjoy it because…,' try 'It is enjoyable because…'")


# And a query intented to prompt a language model to populate the data structure.
require_student = "Do you like to stay in a place with a lot of noise?"

student_answer = """
Oh yes, definitely! There's no denying that living in a place with a lot of noise can make people feel energetic and lively. People enjoy it because it creates a sense of vibrancy and excitement.

However, for me personally, it depends on the situation. For example, I enjoy the buzz of a noisy café when catching up with friends, as it adds to the fun. But when it comes to studying or working, I prefer a quieter environment to help me concentrate better.

Like I said, it really depends on the context and my mood at the time."""

original_text = """
    Oh yes, definitely! There's no denying that living in a noisy place can be vibrant and full of energy. People enjoy it because it keeps them active and engaged.
    
    However, for me personally, it depends on the situation. For example, if I am studying, I prefer a quiet place to concentrate. However, during a festival, I enjoy staying in a lively and noisy environment to feel the energy and excitement.
   
    Like the buzz in a busy café that merges seamlessly into the surroundings, becoming a soundtrack to efficiency and communal activity.
    """

answer2 = """I prefer a quiet place. Noise can be very annoying and I don't like being in loud environments. Sometimes, I go to places with more people, but usually, I avoid it."""

structure_requirements = """Intro: Oh yes, definitely! There's no denying that (positive aspect of something) People enjoy it because (explanations),
    Personal Opinion: However, for me personally, it depends on +  .... + For example, (specific example(s) and explanations),
    Conclusion: Like ..."""
# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=AIAnswer)

prompt = PromptTemplate(
    template="""
    You are an english tutor. With a question require student answer {require_student}, you ask student to write a paragraph including 3 sections (Intro, Personal Opinion, Conclusion) following a strictly structure, each part in (), you should provide detailed feedback for missing part:
    {structure_requirements},
    With a student's answer {student_answer}, you analyze it based on the following objective below:
    - Students need to apply the parts of the structure correctly to construct their answer.
    - AI (you) checks whether students are using the correct structure (including required phrases and sequences).
    Answer the user query.\n{format_instructions}\n""",
    input_variables=["require_student", "student_answer", "structure_requirements"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

res = chain.invoke({"require_student": require_student, "student_answer": student_answer, "structure_requirements": structure_requirements})

In [3]:
res

{'content_score': {'relevance': 0.8, 'logic': 0.9},
 'ai_feedback': 'Great job! You followed the structure perfectly!',
 'missing_example': 'null',
 'missing_intro': 'null',
 'missing_a_short_concluding_sentence': 'null',
 'grammar_errors': 'null',
 'agreement_between_subject_and_verb_errors': 'null',
 'sentence_structure_diversity_errors': 'null'}

In [50]:
res1 = chain.invoke({"require_student": questions, "student_answer": original_text, "structure_requirements": structure_requirements})

In [51]:
res1

{'content_score': {'relevance': 0.8, 'logic': 0.9},
 'ai_feedback': 'Great job! You followed the structure perfectly!',
 'missing_example': None,
 'missing_intro': None,
 'missing_a_short_concluding_sentence': None,
 'grammar_errors': None,
 'agreement_between_subject_and_verb_errors': None,
 'sentence_structure_diversity_errors': None}

In [52]:
res2 = chain.invoke({"require_student": questions, "student_answer": answer2, "structure_requirements": structure_requirements})

In [53]:
res2

{'content_score': {'relevance': 0.6, 'logic': 0.7},
 'ai_feedback': "Your response is clear and understandable. However, avoid minor errors in grammar, like 'being in loud environments' should be 'being in loud environments can be very annoying.' Also, add more specific examples for a stronger answer.",
 'missing_example': "You forgot to include a specific example. Try again with 'For example, noise can be very annoying and disrupt my focus.'",
 'missing_intro': None,
 'missing_a_short_concluding_sentence': None,
 'grammar_errors': ["being in loud environments should be 'being in loud environments can be very annoying.'"],
 'agreement_between_subject_and_verb_errors': None,
 'sentence_structure_diversity_errors': None}

In [54]:
chain = prompt | model | parser
def get_completion(require_student: str, student_answer: str, structure_requirements: str):
    res = chain.invoke({"require_student": require_student, "student_answer": student_answer, "structure_requirements": structure_requirements})
    try:
        return AIAnswer.model_validate(res)
    except Exception as e:
        print(f"Error parsing response: {str(e)}")
        return res

get_completion(require_student, answer2, structure_requirements)

AIAnswer(content_score=ContentScore(relevance=0.6, logic=0.7), ai_feedback="Your response is clear and understandable. However, avoid minor errors in grammar, like 'being in loud environments' should be 'being in loud environments can be very annoying.' Also, add more specific examples for a stronger answer.", missing_example="You forgot to include a specific example. Try again with 'For example, noise from construction sites can make it hard to concentrate.'", missing_intro=None, missing_a_short_concluding_sentence="Don't forget to finish with a short conclusion, like 'Like finding a peaceful place to relax.' to wrap up your answer.", grammar_errors=["being in loud environments should be 'being in loud environments can be very annoying.'"], agreement_between_subject_and_verb_errors=None, sentence_structure_diversity_errors=None)